# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
import cv2 as cv

In [2]:
# params 

original_image_size = (720,1280,3)

# Define the model

In [3]:
# size of the input images
input_image_size = (416,416,3)

# bounding boxes per cell
BB_per_cell = 3 

# number of classes in the dataset
Classes = 5

# Batch size
batch_ = 16

In [4]:
# Weight matrix initialization whenever using relu
initializer_ = tf.keras.initializers.he_normal()

In [5]:
def Build_model(initializer_, input_image_size, BB_per_cell, Classes, batch_, model_name):
    
    # input lauer
    input_layer = tf.keras.layers.Input(shape=input_image_size, batch_size=batch_, name="input")

    # down_sampling
    conv_1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_, padding="same", name="conv_1")(input_layer)

    pooling_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_1")(conv_1)
    
    batch_norm_1 = tf.keras.layers.BatchNormalization()(pooling_1)

    conv_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_,padding="same", name="conv_2")(batch_norm_1)
    
    pooling_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_2")(conv_2)
    
    batch_norm_2 = tf.keras.layers.BatchNormalization()(pooling_2)

    conv_3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_,padding="same", name="conv_3")(batch_norm_2)
    
    pooling_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="pool_3")(conv_3)
    
    batch_norm_3 = tf.keras.layers.BatchNormalization()(pooling_3)

    conv_4 = tf.keras.layers.Conv2D(filters=BB_per_cell*(5+Classes), kernel_size=(3,3), strides=(1,1),
                 activation='relu', kernel_initializer=initializer_,padding="same", name="conv_4")(batch_norm_3)
    
    pooling_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2), name="detection_1")(conv_4)
    
    batch_norm_4 = tf.keras.layers.BatchNormalization()(pooling_4)

    # up sampling

    # up_sampling_1 = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="nearest", name="up_sampling_1")(pooling_4)
    # 
    # concat_layer_1 = tf.keras.layers.Concatenate(name="detection_2")([conv_4,up_sampling_1])
    # 
    # conv_5 = tf.keras.layers.Conv2D(filters=BB_per_cell*(5+Classes), kernel_size=(1,1), strides=(1,1),
    #              activation='relu', kernel_initializer=relu_initializer, name="detection_2")(up_sampling_1)
    # 
    # up_sampling_2 = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation="nearest", name="up_sampling_2")(conv_5)
    # 
    # conv_6 = tf.keras.layers.Conv2D(filters=BB_per_cell*(5+Classes), kernel_size=(1,1), strides=(1,1),
    #              activation='relu', kernel_initializer=relu_initializer, name="detection_3")(up_sampling_2)
    # 
    
    return tf.keras.Model(inputs=input_layer, outputs=[pooling_4], name=model_name)

In [6]:
model = Build_model(initializer_, input_image_size, BB_per_cell, Classes, batch_, "My_SSD_Detector")

In [7]:
model.summary()

Model: "My_SSD_Detector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(16, 416, 416, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (16, 416, 416, 256)       7168      
_________________________________________________________________
pool_1 (MaxPooling2D)        (16, 208, 208, 256)       0         
_________________________________________________________________
batch_normalization (BatchNo (16, 208, 208, 256)       1024      
_________________________________________________________________
conv_2 (Conv2D)              (16, 208, 208, 128)       295040    
_________________________________________________________________
pool_2 (MaxPooling2D)        (16, 104, 104, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (16, 104, 104, 128)   

## Setting up the tensorboard

In [8]:
# set up the tensorbard directory
import os 

root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir(root_logdir):
    import time
    run_dir = time.strftime("run%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_dir)

run_logdir = get_run_logdir(root_logdir)

# tensorboard callback
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)